<h2>Manually inspect images, crop ROI and extract nuclei numbers positive for a cell marker (3D stack to 2D MIP)</h2>

In [1]:
from pathlib import Path
import czifile
import napari
import os
import numpy as np
import pandas as pd
from utils import segment_nuclei_2d, segment_marker_positive_nuclei

<h3>Define the directory where your images are stored</h3>

In [2]:
# Copy the path where your images are stored, ideally inside the raw_data directory
directory_path = Path("./raw_data/test_data")

# Create an empty list to store all image filepaths within the dataset directory
images = []

# Create an empty list to store all stats extracted from each image
stats = []

# Iterate through the .czi files in the directory
for file_path in directory_path.glob("*.czi"):
    images.append(str(file_path))
    
images

['raw_data\\test_data\\HI 1  Contralateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1.czi',
 'raw_data\\test_data\\HI 1  Ipsilateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1.czi']

<h3>Open each image in the directory</h3>
You can do so by changing the number within the brackets below <code>image = images[0]</code>

In [3]:
# Explore a different image to crop (0 defines the first image in the directory)
image = images[0]

# Read path storing raw image and extract filename
file_path = Path(image)
filename = file_path.stem

# Read the image file
img = czifile.imread(image)

# Remove singleton dimensions
img = img.squeeze()

# Perform MIP on all channels
img_mip = np.max(img, axis=1)

# Show image in Napari to define ROI
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img_mip)

# Feedback for researcher
print(f"Image displayed: {filename}")
print(f"Array shape: {img.shape}")
print(f"MIP Array shape: {img_mip.shape}")

Invalid schema for package 'ome-types', please run 'npe2 validate ome-types' to check for manifest errors.


Image displayed: HI 1  Contralateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1
Array shape: (4, 14, 3803, 2891)
MIP Array shape: (4, 3803, 2891)


<h3>Crop your regions of interest</h3>

If you are using complex polygons instead of a rectangle to define your ROI, give Napari some time to perform the cropping. Then come back and continue running the analysis pipeline

<video controls>
  <source src="./assets/napari_crop.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>

In [4]:
# Code snippet to analyze cropped regions (ROIs) instead of the full image

try:
    # Access the specific layer by name
    cropped_layer = viewer.layers['img cropped [0]']

    # Get the image data as a NumPy array
    img_mip = cropped_layer.data
    print("ROI selected.")

    # Store info about cropping
    crop = True
    
except:
    # Store info about cropping
    crop = False
    print("No ROI selected. Whole image will be analyzed.")


No ROI selected. Whole image will be analyzed.


<h3>Define your nuclei/marker stacks and your marker/erosion thresholds</h3>

Modify the values for <code>nuclei_channel</code>, <code>marker_channel</code>, <code>nuclei_channel_threshold</code> and <code>erosion_factor</code>

In [5]:
# Define the nuclei and markers of interest channel order ('Remember in Python one starts counting from zero')
nuclei_channel = 3
marker_channel = 1

# Define the intensity threshold above which a cell is considered positive for a marker
marker_channel_threshold = 40

# Sets the amount of erosion that is applied to areas where the marker+ signal colocalizes with nuclear signal
# The higher the value, the stricter the conditions to consider a nuclei as marker+
erosion_factor = 3

# Slice the nuclei and marker stack
nuclei_img = img_mip[nuclei_channel, :, :]
marker_img = img_mip[marker_channel, :, :]

# Segment nuclei and return labels
nuclei_labels = segment_nuclei_2d_mip(nuclei_img)

# Select marker positive nuclei
marker_mip, processed_region_labels = segment_marker_positive_nuclei (nuclei_labels, marker_img, marker_channel_threshold, erosion_factor)

<h3>Visualize the results in Napari</h3>

In [7]:
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img_mip)
viewer.add_image(nuclei_img)
viewer.add_labels(nuclei_labels)
# viewer.add_image(marker_mip)
viewer.add_labels(processed_region_labels, name=f"marker+_nuclei")

<Labels layer 'marker+_nuclei' at 0x244a8fe5af0>

<h3>Data extraction</h3>


In [ ]:
# Define output folder for results
results_folder = "./results/"

# Create the necessary folder structure if it does not exist
try:
    os.mkdir(str(results_folder))
    print(f"Output folder created: {results_folder}")
except FileExistsError:
    print(f"Output folder already exists: {results_folder}")

# Extract your information of interest
total_nuclei = len(np.unique(processed_region_labels)) - 1
marker_pos_nuclei = len(np.unique(nuclei_labels)) - 1

# Create a dictionary containing all extracted info per image
stats_dict = {
            "filename": filename,
            "cropped": crop,
            "total_nuclei": total_nuclei,
            "marker+_nuclei": marker_pos_nuclei,
            "%_marker+_cells": (total_nuclei * 100) / marker_pos_nuclei
            }

# Append the current data point to the stats_list
stats.append(stats_dict)

# Transform into a dataframe to store it as .csv later
df = pd.DataFrame(stats)

# Overwrite the .csv with new data points each round
df.to_csv("./results/marker_+_manual_2D.csv", index=True)

df